In [1]:
import pandas as pd
import psycopg2

Set up a connection to the database

In [ ]:
connection = psycopg2.connect(host = <aws_endpoint>,
                              port = <port_number>,
                              user = <master_username>
                              password = <master_password>,
                              dbname = <database_name>)

cursor = connection.cursor()

Helper functions for reading and writing from database

In [3]:
def read_try(sql):
    try:
        df = pd.read_sql(sql, con=connection)
        return pd.DataFrame() if df.empty else df
    except Exception as e:
        print("READ ERROR", e)
        return pd.DataFrame()

def write_try(sql):
    try:
        cursor.execute(sql)  # run a psql command
        return True
    except Exception as e:
        print("WRITE ERROR: ", e)
        return False
    finally:
        connection.commit()

Create _company_ and _employee_ tables.

In [4]:
sql = '''CREATE TABLE company (
   name VARCHAR(280) PRIMARY KEY
);'''
write_try(sql)

sql = '''CREATE TABLE employee (
   id SERIAL,
   company_name VARCHAR(280) NOT NULL,
   PRIMARY KEY (id, company_name),
   name VARCHAR(280),
   role VARCHAR(280),
   CONSTRAINT company_name_fkey FOREIGN KEY (company_name)
        REFERENCES company (name) MATCH SIMPLE
        ON UPDATE NO ACTION ON DELETE NO ACTION
);'''
write_try(sql)

True

Write records to tables. 

In [5]:
sql = '''INSERT INTO company VALUES
                           ('headbook'),
                           ('doogle');'''
write_try(sql)

sql = '''INSERT INTO employee (company_name, name, role)
VALUES
       ('headbook', 'John Smith', 'CEO'),
       ('headbook', 'Jane Doe', 'VP of Technology'),
       ('doogle', 'Jane Doe', 'CEO'),
       ('doogle', 'John Smith', 'VP of Technology');'''
write_try(sql)

True

Read from tables. 

In [6]:
sql = '''SELECT * FROM company;'''
read_try(sql)

,name
0,headbook
1,doogle


In [7]:
sql = '''SELECT * FROM employee;'''
read_try(sql)

,id,company_name,name,role
0,1,headbook,John Smith,CEO
1,2,headbook,Jane Doe,VP of Technology
2,3,doogle,Jane Doe,CEO
3,4,doogle,John Smith,VP of Technology


If you want to delete the table, delete the _employee_ table first, since there are constraints on the _employee_ table to the _company_ table. Then delete the _company_ table.

In [8]:
sql = '''DROP TABLE employee'''
write_try(sql)

sql = '''DROP TABLE company'''
write_try(sql)

True